<img src="http://openenergy-platform.org/static/OEP_logo_2_no_text.svg" alt="OpenEnergy Platform" height="100" width="100"  align="left"/>


# OpenEnergy Platform
<br><br>

# Tutorial Create a Dataframe with geopackage export
Repository: https://github.com/OpenEnergyPlatform/tutorial

Please report bugs and improvements here: https://github.com/OpenEnergyPlatform/examples/issues <br>
How to get started with Jupyter Notebooks can be found here: https://github.com/OpenEnergyPlatform/oeplatform/wiki

In [ ]:
__copyright__ = "Reiner Lemoine Institut, Zentrum für nachhaltige Energiesysteme Flensburg"
__license__   = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__       = "https://github.com/openego/data_processing/blob/master/LICENSE"
__author__    = "TuPhanRLI"

## Introduction
<br>
<div class="alert alert-block alert-danger">
This is an important information!
</div>
<div class="alert alert-block alert-info">
This is an information!
</div>
<div class="alert alert-block alert-success">
This is your task!
</div>

This tutorial gives you an overview of the [**OpenEnergy Platform**](http://oep.iks.cs.ovgu.de/) and how you can work with the **REST-full-HTTP** API in Python. <br>
The full API documentaion can be found on [ReadtheDocs.io](http://oep-data-interface.readthedocs.io/en/latest/api/how_to.html#authenticate "OpenEnergyPlatform’s documentation").


## Content

0 Setup token <br>
1 Select data <br>
2 Make a pandas dataframe <br>
3 Plot a dataframe (geo plot)<br>
4 Save data 

## 0. Setup token
<div class="alert alert-block alert-info">
The token is used to verify the API interaction for your OEP user.
</div>
<div class="alert alert-block alert-danger">
Do not push your token to GitHub!
</div>

In [ ]:
import requests
import pandas as pd
from IPython.core.display import HTML

from token_config import oep_url, get_oep_token

# token
your_token = get_oep_token()

## 1. Select data

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import shapely.wkt
from shapely import wkb
from geoalchemy2.shape import to_shape

In [ ]:
# select data
schema = 'openstreetmap'
table = 'osm_deu_point_windpower'
requested_data = requests.get(oep_url+'/api/v0/schema/'+schema+'/tables/'+table+'/rows', )
requested_data.status_code

<div class="alert alert-block alert-info">
<b>Response [200]</b> succesfully selected data! <br>
<b>Response [404]</b> table doesn't exist!
</div>

## 2. Make a pandas dataframe

<div class="alert alert-block alert-success">
Create pandas dataframes for each data set returned as API result!
</div>

In [ ]:
#Create dataframe from json format
df = pd.DataFrame(requested_data.json())

<div class="alert alert-block alert-success">
Let's take a look into our data!
</div>

In [ ]:
# Show metadata for a specific dataframe.
df.info()

In [ ]:
#Print the df_pp dataframe as table.
df.head()

In [ ]:
#visualization of the dataframe 
import missingno 
missingno.bar(df, color='tab:blue');

## 3. Plot a dataframe (geo plot)

In [ ]:
import geopandas as gpd
import shapely
import matplotlib.pyplot as plt
%matplotlib inline

<div class="alert alert-block alert-success">
If we want to apply a change to every entity in a column we can use Pandas apply function.
</div>

In [ ]:
#Print the df geodataframe as table with geometry data
df.geom.head()

In [ ]:
# transform WKB to WKT / Geometry specially the geom column
df['geom'] = df['geom'].apply(lambda x:shapely.wkb.loads(x, hex=True))

In [ ]:
#Print the gdf geodataframe as table with geometry data
df.geom.head()

<div class="alert alert-block alert-success">
Let's plot our data!
</div>

</p>crs parameters can be changd depends on your source and location.</p> <br>
</p>At the following lines there are possibilities to set up the crs variable.</p> <br>
</p>WGS84 Latitude/Longitude: "EPSG:4326"</p> <br>
</p>UTM Zones (North): "EPSG:32633"</p> <br>
</p>UTM Zones (South): "EPSG:32733"</p> <br>

In [ ]:
# geo plot data
crs = {'init' :'epsg:32633'}
gdf = gpd.GeoDataFrame(
                        df,# specifify your dataframe
                        crs=crs, # this is your coordinate system
                        geometry=df.geom # specify the geometry list we created
                        )
base1 = gdf.plot(color='white', edgecolor='black',figsize=(16,16))
gdf.plot(ax=base1, color='tab:blue')
plt.show()

In [ ]:
# Show metadata for a specific (geo)dataframe.
gdf.info()

## 4. Save Data

In [ ]:
# Convert the GeoDataFrame to GeoPackage and GeoJSON Format and save file 
# at the folder path "output_GeoData"
gdf.geometry.to_file("output_GeoData/example_geo_json.geojson", driver='GeoJSON')
gdf.geometry.to_file("output_GeoData/example_geopackage.gpkg", layer='data_example', driver="GPKG")